## Notebook to compute total demand by TAZ for the transit mode

In [ ]:
# Notebook to compute total demand by TAZ for the transit mode

import openmatrix as omx
import numpy as np
import pandas as pd
import geopandas as gp
import matplotlib.pyplot as plt
import bokeh
import xarray as xr
import hvplot.pandas
import hvplot.xarray

In [ ]:
%matplotlib notebook

In [ ]:
# Root directory for MoDX output for "base year" model results.
#
base_scenario_dir = r'G:/Regional_Modeling/1A_Archives/LRTP_2018/2016 Scen 00_08March2019_MoDXoutputs/'
#
# Root directory for MoDX output for "comparison scenario" model results.
# 
comparison_scenario_dir = r'G:/Regional_Modeling/1A_Archives/LRTP_2018/2040 NB Scen 01_MoDXoutputs/'

### User input required: Specify path to root directory of scenario to use

In [ ]:
# ===>>> USER INPUT REQUIRED: <<<===
#
# Supply path to root directory of scenario to use for the current run of this notebook:
# 
home_dir = comparison_scenario_dir

### User input required: Specify CSV report output file

In [ ]:
# ===>>> USER INPUT REQUIRED: <<<===
#
# 2. Supply path to root of user's "sandbox" directory:
#
my_sandbox_dir = r'S:/my_modx_output_dir/'
#
# 3. Supply name of CSV output file for tabular results generated by this notebook:
#
csv_output_fn = 'taz_transit_report_comp.csv'

In [ ]:
taz_shapefile_base_dir = r'G:/Data_Resources/modx/canonical_TAZ_shapefile/'

In [ ]:
# trip_tables directory - this really "should" be a subdirectory of the base directory, but is isn't currently.
# The real McCoy - where things should go, and will eventually go
tt_dir = home_dir + 'out/'

In [ ]:
# trip tables OMX file (matrices)
tt_am = tt_dir + 'AfterSC_Final_AM_Tables.omx'
tt_md = tt_dir + 'AfterSC_Final_MD_Tables.omx'
tt_pm = tt_dir + 'AfterSC_Final_PM_Tables.omx'
tt_nt = tt_dir + 'AfterSC_Final_NT_Tables.omx'
trip_tables = { 'am' : omx.open_file(tt_am, 'r'),
                'md' : omx.open_file(tt_pm, 'r'),
                'pm' : omx.open_file(tt_pm,'r'),
                'nt' : omx.open_file(tt_nt, 'r') }

In [ ]:
num_tazes = trip_tables['am'].shape()[0]

In [ ]:
# Mapping from TAZ-ID to OMX index for the 4 periods (these *should* be the same)
taz_to_omxid_am = trip_tables['am'].mapping('ID')
taz_to_omxid_am = trip_tables['md'].mapping('ID')
taz_to_omxid_pm = trip_tables['pm'].mapping('ID')
taz_to_omxid_nt =  trip_tables['nt'].mapping('ID')

In [ ]:
# We'll assume that the mapping from TAZ ID to OMX ID doesn't vary by time period.
# We'll use the AM mapping as _the_ mapping for all time periods, pending confirmation.
# 
# TBD: Insert "sanity check" that the 4 mappings on "ID" are identical.
#
taz_to_omxid = taz_to_omxid_am

In [ ]:
# Function: load_tts_for_mode_list_as_np_arrays
#
# Summary: Load all trip tables for all time periods for a specified list of modes as NumPy arrays,
# and return a two-level dictionary (i.e., by time period and by mode) of the results.
#
def load_tts_for_mode_list_as_np_arrays(tts, mode_list):
    all_periods_list = all_periods_list = ['am', 'md', 'pm', 'nt']
    retval = {'am' : None, 'md' : None, 'pm' : None, 'nt' : None }
    for period in all_periods_list:
        retval[period] = {}
        for mode in mode_list:
            temp = tts[period][mode]
            retval[period][mode] = np.array(temp)
        # end_for
    # end_for
    return retval
# end_def load_tts_for_mode_list_as_np_arrays()

### Load trip tables for transit mode

In [ ]:
# Load trip tables for transit mode
temp = load_tts_for_mode_list_as_np_arrays(trip_tables, \
        [ 'DAT_Boat', 'DET_Boat', 'DAT_CR', 'DET_CR', 'DAT_LB', 'DET_LB', 'DAT_RT', 'DET_RT', 'WAT'])

In [ ]:
# Compute total Boat, Commuter Rail (CR), Local Bus (LB), Rapid Transit (RT), Walk-access and total transit demand from each TAZ
#
boat_dat_temp = temp['am']['DAT_Boat'] + temp['md']['DAT_Boat'] + temp['pm']['DAT_Boat'] + temp['nt']['DAT_Boat']
boat_dat_demand = boat_dat_temp.sum(axis=1)
boat_det_temp = temp['am']['DET_Boat'] + temp['md']['DET_Boat'] + temp['pm']['DET_Boat'] + temp['nt']['DET_Boat']
boat_det_demand = boat_det_temp.sum(axis=1)
boat_demand_total = boat_dat_demand + boat_det_demand
#
cr_dat_temp = temp['am']['DAT_CR'] + temp['md']['DAT_CR'] + temp['pm']['DAT_CR'] + temp['nt']['DAT_CR']
cr_dat_demand = cr_dat_temp.sum(axis=1)
cr_det_temp = temp['am']['DET_CR'] + temp['md']['DET_CR'] + temp['pm']['DET_CR'] + temp['nt']['DET_CR']
cr_det_demand = cr_det_temp.sum(axis=1)
cr_demand_total = cr_dat_demand + cr_det_demand
#
lb_dat_temp = temp['am']['DAT_LB'] + temp['md']['DAT_LB'] + temp['pm']['DAT_LB'] + temp['nt']['DAT_LB']
lb_dat_demand = lb_dat_temp.sum(axis=1)
lb_det_temp = temp['am']['DET_LB'] + temp['md']['DET_LB'] + temp['pm']['DET_LB'] + temp['nt']['DET_LB']
lb_det_demand = lb_det_temp.sum(axis=1)
lb_demand_total = lb_dat_demand + lb_det_demand
#
rt_dat_temp = temp['am']['DAT_RT'] + temp['md']['DAT_RT'] + temp['pm']['DAT_RT'] + temp['nt']['DAT_RT']
rt_dat_demand = rt_dat_temp.sum(axis=1)
rt_det_temp = temp['am']['DET_RT'] + temp['md']['DET_RT'] + temp['pm']['DET_RT'] + temp['nt']['DET_RT']
rt_det_demand = rt_det_temp.sum(axis=1)
rt_demand_total = rt_dat_demand + rt_det_demand
#
wat_temp = temp['am']['WAT'] + temp['md']['WAT'] + temp['pm']['WAT'] + temp['nt']['WAT']
wat_demand_total = wat_temp.sum(axis=1)
# 
total_transit_demand = boat_demand_total + cr_demand_total + lb_demand_total + rt_demand_total + wat_demand_total

In [ ]:
# Build dataframes of Boat, Commuter Rail, Local Bus, Rapid Transit, walk-access, and total transit demand from each TAZ
# Set each data frame's index to the omxid of each row, i.e., its index
#
boat_df = pd.DataFrame(boat_demand_total, columns=['boat'])
boat_df['omxid'] = boat_df.index
boat_df.set_index('omxid')
#
cr_df = pd.DataFrame(cr_demand_total, columns=['cr'])
cr_df['omxid'] = cr_df.index
cr_df.set_index('omxid')
#
lb_df = pd.DataFrame(lb_demand_total, columns=['lb'])
lb_df['omxid'] = lb_df.index
lb_df.set_index('omxid')
#
rt_df = pd.DataFrame(rt_demand_total, columns=['rt'])
rt_df['omxid'] = rt_df.index
rt_df.set_index('omxid')
#
wat_df = pd.DataFrame(wat_demand_total, columns=['wat'])
wat_df['omxid'] = wat_df.index
wat_df.set_index('omxid') 
#
transit_df = pd.DataFrame(total_transit_demand, columns=['transit_total'])
transit_df['omxid'] = wat_df.index
transit_df.set_index('omxid')

In [ ]:
# Join the 6 dataframes into a single data frame
temp1_df = pd.merge(left=transit_df, right=boat_df, on="omxid")
temp2_df = pd.merge(left=temp1_df, right=cr_df, on="omxid")
temp3_df = pd.merge(left=temp2_df, right=lb_df, on="omxid")
temp4_df = pd.merge(left=temp3_df, right=rt_df, on="omxid")
total_transit_trips_df = pd.merge(left=temp4_df, right=wat_df, on="omxid")

In [ ]:
# Load the candidate canonical TAZ shapefile as a geopands dataframe.
# N.B. Use shapefile in WGS84 SRS.
#
taz_shapefile = taz_shapefile_base_dir + 'candidate_CTPS_TAZ_STATEWIDE_2019_wgs84.shp'
taz_gdf = gp.read_file(taz_shapefile)
taz_gdf.set_index("id")

In [ ]:
# Add a 'omxid' column to the TAZ geodataframe, in prep for joining with the total trips dataframes.
# ==> This also can be done earlier.
taz_gdf['omxid'] = taz_gdf.apply(lambda row: taz_to_omxid[row.id], axis=1)

In [ ]:
# Join the shapefile geodataframe to the total trips dataframe on 'omxid'
joined_df = taz_gdf.join(total_transit_trips_df.set_index('omxid'), on='omxid')

### Export report output to CSV file

In [ ]:
# Export the useful columns of data in the 'joined_df' dataframe as a CSV file
fq_output_fn = my_sandbox_dir + csv_output_fn
joined_df.to_csv(fq_output_fn, sep=',', 
                columns=['id', 'town', 'state', 'transit_total', 'boat', 'cr', 'lb', 'rt', 'wat'])

### Generate static and interactive maps of results

In [ ]:
# Make a static map of total auto trips by origin TAZ
joined_df.plot("transit_total", figsize=(10.0,8.0), cmap='plasma', legend=True)
plt.title('Total Transit Trips by Origin TAZ')
plt.show()

In [ ]:
# Make an interactive map of the above
joined_df.hvplot(c='transit_total', 
                 geo=True, 
                 hover_cols=['id', 'town', 'transit_total', 'boat', 'cr', 'lb', 'rt', 'wat'], 
                 clabel='Total Transit Trips', 
                 cmap='plasma',
                 frame_height=500).opts(title='Total Transit Trips by Origin TAZ')